# pre processing all current raw data for action id process (Step 4, 5,6)
    steps:
    1 - Organize all files by user (with related information)
    2 - merge all related files per user
    3 - sort by timestamp
    4 - **decide the certain "action key"(exp. 1:login 2:longoff)**
    5 - **break the log into action keys (origional using auto-generated non-repeat num ID with parsed log type)**
    6 - **new line while "logoff" (origional using block_id which not exist here)**

In [21]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

LDAP_dir = "./r5.2/LDAP"
Processed_dir = "./pre_processed"
result_dir = "./processed_log"
temp_test = "./test_final_raw.csv"

In [22]:
#"action key" the element of LogKey
ACTION_DIR = {"Logon": "1",
              "Logoff": "2",
              "Connect": "3",
              "Disconnect": "4",
              "http": "5",
              "file": "6",
              "email": "7"}

In [23]:
ACTION_DIR["email"]

'7'

test with one user

In [24]:
test_file = pd.read_csv(temp_test)
test_file
# test_file.shape()

,Unnamed: 0,date,action
0,0,01/01/2010 08:04:00,Logon
1,1,01/01/2010 20:03:00,Logoff
2,2,01/02/2010 07:53:00,Logon
3,3,01/02/2010 12:18:06,Logon
4,4,01/02/2010 20:02:00,Logoff
5,5,01/04/2010 08:04:00,Logon
6,6,01/04/2010 20:08:00,Logoff
7,7,01/05/2010 08:02:00,Logon
8,8,01/05/2010 13:41:00,Logon
9,9,01/05/2010 19:55:00,Logoff


In [25]:
test_file = test_file.drop(columns=["Unnamed: 0"])
test_file

,date,action
0,01/01/2010 08:04:00,Logon
1,01/01/2010 20:03:00,Logoff
2,01/02/2010 07:53:00,Logon
3,01/02/2010 12:18:06,Logon
4,01/02/2010 20:02:00,Logoff
5,01/04/2010 08:04:00,Logon
6,01/04/2010 20:08:00,Logoff
7,01/05/2010 08:02:00,Logon
8,01/05/2010 13:41:00,Logon
9,01/05/2010 19:55:00,Logoff


In [26]:
test_file = test_file.sort_values(by=['date'])

In [27]:
with open("./test_logKey.txt","w") as out_file:
    for _, row in tqdm(test_file.iterrows()):
        out_file.writelines(ACTION_DIR[row.action]+' ')
        if row.action == "Logoff":
            out_file.writelines('\n')

29543it [00:02, 14200.26it/s]
